In [1]:
%matplotlib inline
import time
import numpy as np
import xesmf as xe
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import cartopy.crs as ccrs
import matplotlib.ticker
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import warnings
warnings.filterwarnings('ignore')

from numba import stencil

In [2]:
start_time = time.time()

In [3]:
@stencil
def coast(a):
    return a[0,0] & ~(a[0,1] & a[0,-1] & a[1,0] & a[-1,0])

In [4]:
d = xr.open_dataset("/scratch/nm03/lxy581/mom6/archive/tides_010_global_sigma_SAL_x06/output003/ocean_static.nc")
# print(coast(d.wet.data.astype(int)))

In [5]:
coast_mask = coast(d.wet.data.astype(int))
coast_mask_u = coast(d.wet_u.data.astype(int))
coast_mask_v = coast(d.wet_v.data.astype(int))

In [6]:
print(np.sum(coast_mask))
print(np.sum(coast_mask_u))
print(np.sum(coast_mask_v))

77680
76265
76231


In [7]:
print(coast_mask.shape)
print(coast_mask_u.shape)
print(coast_mask_v.shape)

(2784, 3600)
(2784, 3601)
(2785, 3600)


In [8]:
data = xr.open_dataset("/g/data/nm03/lxy581/tpxo9v5a/TPXO_M2_P_10km.nc")

In [9]:
etaH_u = np.array(data.etaH_u)
etaH_v = np.array(data.etaH_v)
lat_v = np.array(data.lat_v)/180 *np.pi

In [10]:
out = xr.open_dataset('/scratch/nm03/lxy581/mom6/archive/tides_010_global_sigma_SAL_x06/output002/ocean_interior.nc')
time_len = 236
u = out.uo.isel(zl=0, time=slice(None, time_len))
v = out.vo.isel(zl=0, time=slice(None, time_len))

In [11]:
veta = np.nanmean(v * etaH_v * np.cos(lat_v), axis=0) 
ueta = np.nanmean(u * etaH_u, axis=0)

In [12]:
end_time = time.time()
exe_time = float(end_time - start_time)
print("Execution time: %.1f mins!" % (exe_time/60.0))

Execution time: 4.4 mins!


In [13]:
v_mean = np.nanmean(v, axis=0) 
u_mean = np.nanmean(u, axis=0)

In [14]:
end_time = time.time()
exe_time = float(end_time - start_time)
print("Execution time: %.1f mins!" % (exe_time/60.0))

Execution time: 5.6 mins!


In [15]:
print(veta.shape)
print(ueta.shape)

(2785, 3600)
(2784, 3601)


In [16]:
veta_mask = veta * coast_mask_v
ueta_mask = ueta * coast_mask_u

In [17]:
veta_sum = np.nansum(veta_mask)
ueta_sum = np.nansum(ueta_mask)

In [18]:
print(veta_sum)
print(ueta_sum)

11443.659900483326
-18273.9609739176


In [19]:
v_mask = v_mean * coast_mask_v
u_mask = u_mean * coast_mask_u

In [20]:
v_sum = np.nansum(v_mask)
u_sum = np.nansum(u_mask)

In [21]:
print(v_sum)
print(u_sum)

11.607115779747472
6.493346444972548


In [22]:
end_time = time.time()
exe_time = float(end_time - start_time)
print("Execution time: %.1f mins!" % (exe_time/60.0))

Execution time: 5.6 mins!
